In [4]:
from dotenv import load_dotenv
load_dotenv("/home/wpnx/.secrets/.env")

True


### Readme 
I created the chatbot using the provided documents, but it is not answering questions about CIS Rules correctly
because no documents were provided for CIS Rules. However, I have added some websites that contain information 
on CIS Rules. (We will use web scraping to extract the data from these websites, which will then be saved in the vector database.)

With CIS Documents llm model could have run better


In [67]:
#importing libaries
from langchain_community.document_loaders import DirectoryLoader, WebBaseLoader, UnstructuredExcelLoader
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings,SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
import pandas as pd
import requests
from langchain.llms import Ollama
from langchain.agents import tool, initialize_agent,AgentExecutor, create_react_agent
from langchain import hub

In [133]:
### Here we are using gemma2:2b model
# model size is 1.6 gb so it could run on cpu also

In [126]:
# llm = ChatGroq(name="llama3.1")
llm = Ollama(model= "gemma2:2b")

In [115]:
### setting up llm and embedding model
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/wpnx/CODES/ENVS/LLM_BASE/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [127]:
#creaing Vectordatabase

splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=150)
web_loader = WebBaseLoader([ "https://www.prnewswire.com/news-releases/center-for-internet-security-cis-releases-cis-controls-v8-to-reflect-evolving-technology-threats-301293567.html",
                           "https://www.cisecurity.org/terms-and-conditions-table-of-contents",
                            "https://www.cisecurity.org/controls/cis-controls-list"])
temp_docs = web_loader.load()
docs = splitter.split_documents(temp_docs)
cis_db = FAISS.from_documents(docs, embed_model)

## Excel Documents
splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=150)
excel_loader = DirectoryLoader("Datasets/excel_docs/",glob="**/*.xlsx")
excel_docs = splitter.split_documents(excel_loader.load())
Excel_Database = FAISS.from_documents(excel_docs, embed_model)

In [94]:
main_docs = []

In [95]:
main_docs.extend(docs)

In [96]:
len(main_docs)

55

In [97]:
main_docs.extend(excel_docs)

In [98]:
len(main_docs)

1327

In [99]:
main_database = FAISS.from_documents(main_docs, embed_model)

In [134]:
### this is main database whcih conatains info about CIS, AWS, GCP

In [128]:
## Prompt for CIS and AWS GCP chain
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

cis_template = """
You are a bot for Center for Internet Security (CIS). You have to 
answer CIS Questions. If you dont know answer just say i dont know
Question:{question}
Context:{context}
Answer
"""


env_temp="""
You are helful bot for Center for Internet Security (CIS), Amazons AWS and Googles GCP cloud Information. which answers users query from the give context.
From given context find the answers if answer not in the context just say I dont know , nothing else.
Question:{question}
Context:{context}
Answer:
"""

env_prompt = PromptTemplate.from_template(env_temp)
# agent_prompt = hub.pull("hwchase17/react")
# cis_prompt = PromptTemplate.from_template(cis_template)



In [129]:
from langchain.chains import RetrievalQA

env_chain = RetrievalQA.from_chain_type(llm=llm,retriever=main_database.as_retriever() ,chain_type_kwargs={"prompt":env_prompt})
# cis_chain = RetrievalQA.from_chain_type(llm=llm,retriever=cis_db.as_retriever() ,chain_type_kwargs={"prompt":cis_prompt},return_source_documents=True)

In [130]:
def chat(question):
    answer = env_chain.invoke(question)["result"]
    return {"question":question, "answer":answer}

In [120]:
### LLM Rsult

In [123]:
chat("How do I apply the CIS rule to disable unused services on a Linux server?")

{'question': 'How do I apply the CIS rule to disable unused services on a Linux server?',
 'answer': "The context provided does not include information on how to apply the CIS rule to disable unused services on a Linux server. It includes controls and descriptions related to uninstalling or disabling unnecessary services, ensuring only approved ports, protocols, and services are running, and denying communication over unauthorized ports. However, it does not provide specific steps or instructions for implementing these controls on a Linux server. Therefore, my answer is: I don't know."}

In [124]:
chat("Can you tell me what the CIS compliance guidelines for secure password policies are?")

{'question': 'Can you tell me what the CIS compliance guidelines for secure password policies are?',
 'answer': 'The CIS compliance guidelines for secure password policies recommend that the IAM password policy requires a minimum length of 14 or greater and prevents password reuse. Setting a password complexity policy and preventing password reuse increases account resiliency against brute force login attempts.'}

In [125]:
chat("What is CIS Rule 99.9.9?")

{'question': 'What is CIS Rule 99.9.9?',
 'answer': "I don't know. The context provided does not include information about CIS Rule 99.9.9."}

In [131]:
chat("What is CIS Rule 99.9.9?")

{'question': 'What is CIS Rule 99.9.9?', 'answer': 'I dont know \n'}